In [1]:
import pandas as pd
from dotenv import load_dotenv

import os
import json
import requests
from datetime import datetime

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
BUILDKITE_API_TOKEN = os.getenv('BUILDKITE_API_TOKEN')
ORGANIZATION_SLUG='vllm'
PIPELINE_SLUG = 'ci-aws'
TODAY = datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z') # it is UTC, so -2 hours from Finnish local time


In [4]:
# Set the URL
url = f"https://api.buildkite.com/v2/organizations/{ORGANIZATION_SLUG}/pipelines/{PIPELINE_SLUG}/builds"

# Set the query parameters
params = {
    'created_from': TODAY,
}

# Set the headers
headers = {
    'Authorization': f'Bearer {BUILDKITE_API_TOKEN}'
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    df = pd.json_normalize(data)
else:
    print(f"Request failed with status code {response.status_code}")

In [5]:
df.head()

,id,graphql_id,url,web_url,number,state,cancel_reason,blocked,blocked_state,message,commit,branch,tag,source,creator,created_at,scheduled_at,started_at,finished_at,pull_request,rebuilt_from,jobs,cluster_id,cluster_url,author.username,author.name,author.email,meta_data.buildkite:git:commit,pipeline.id,pipeline.graphql_id,pipeline.url,pipeline.web_url,pipeline.name,pipeline.description,pipeline.slug,pipeline.repository,pipeline.cluster_id,pipeline.pipeline_template_uuid,pipeline.branch_configuration,pipeline.default_branch,pipeline.skip_queued_branch_builds,pipeline.skip_queued_branch_builds_filter,pipeline.cancel_running_branch_builds,pipeline.cancel_running_branch_builds_filter,pipeline.allow_rebuilds,pipeline.provider.id,pipeline.provider.settings.build_branches,pipeline.provider.settings.build_merge_group_checks_requested,pipeline.provider.settings.build_pull_request_base_branch_changed,pipeline.provider.settings.build_pull_request_forks,pipeline.provider.settings.build_pull_request_labels_changed,pipeline.provider.settings.build_pull_request_ready_for_review,pipeline.provider.settings.build_pull_requests,pipeline.provider.settings.build_tags,pipeline.provider.settings.cancel_deleted_branch_builds,pipeline.provider.settings.filter_enabled,pipeline.provider.settings.prefix_pull_request_fork_branch_names,pipeline.provider.settings.publish_blocked_as_pending,pipeline.provider.settings.publish_commit_status_per_step,pipeline.provider.settings.publish_commit_status,pipeline.provider.settings.pull_request_branch_filter_enabled,pipeline.provider.settings.separate_pull_request_statuses,pipeline.provider.settings.skip_builds_for_existing_commits,pipeline.provider.settings.skip_pull_request_builds_for_existing_commits,pipeline.provider.settings.trigger_mode,pipeline.provider.settings.use_step_key_as_commit_status,pipeline.provider.settings.repository,pipeline.provider.settings.pull_request_branch_filter_configuration,pipeline.provider.settings.filter_condition,pipeline.provider.webhook_url,pipeline.builds_url,pipeline.badge_url,pipeline.created_by.id,pipeline.created_by.graphql_id,pipeline.created_by.name,pipeline.created_by.email,pipeline.created_by.avatar_url,pipeline.created_by.created_at,pipeline.created_at,pipeline.archived_at,pipeline.env,pipeline.scheduled_builds_count,pipeline.running_builds_count,pipeline.scheduled_jobs_count,pipeline.running_jobs_count,pipeline.waiting_jobs_count,pipeline.visibility,pipeline.tags,pipeline.emoji,pipeline.color,pipeline.configuration,pipeline.steps,pipeline.cluster_url,creator.id,creator.graphql_id,creator.name,creator.email,creator.avatar_url,creator.created_at,pull_request.id,pull_request.base,pull_request.repository,author,rebuilt_from.id,rebuilt_from.number,rebuilt_from.url
0,0192f6ab-038e-4e77-983b-b36f78e9a955,QnVpbGQtLS0wMTkyZjZhYi0wMzhlLTRlNzctOTgzYi1iMz...,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683,10683,running,None,False,,[Bugfix][OpenVINO] Fix circular reference #993...,ccb5376a9a88bb6251c4434b79c173151e6f7729,main,None,webhook,NaN,2024-11-04T10:14:15.758Z,2024-11-04T10:14:15.683Z,2024-11-04T10:14:53.296Z,None,NaN,NaN,[{'id': '0192f6ab-03cc-424b-800c-0b85592bf40c'...,None,None,MengqingCao,Mengqing Cao,cmq0113@163.com,commit ccb5376a9a88bb6251c4434b79c173151e6f772...,018fac60-6fa2-4681-9714-0ffcf8e619e9,UGlwZWxpbmUtLS0wMThmYWM2MC02ZmEyLTQ2ODEtOTcxNC...,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws,CI AWS,CI testing of vLLM repo running on AWS,ci-aws,https://github.com/vllm-project/vllm.git,None,None,main,main,True,!main,True,!main,True,github,True,False,False,True,True,False,True,False,False,True,True,False,True,True,False,True,False,True,code,False,vllm-project/vllm,,"(build.pull_request.labels includes ""ready"" &&...",https://webhook.buildkite.com/deliver/3e7ddff5...,https://api.buildkite.com/v2/organizations/vll...,https://badge.buildkite.com/0f7dfe72085fa6fc9e...,018d3825-2689-4a67-9591-

In [6]:
df.shape

(22, 106)

In [7]:
df.state.value_counts()

state
passed      10
failed       6
canceled     4
running      1
failing      1
Name: count, dtype: int64

In [43]:
useful_columns = ['id', 'web_url', 'url', 'number', 'state', 'cancel_reason', 'blocked', 'blocked_state', 'jobs']

In [44]:
d = df[useful_columns]

In [45]:
d.head()

,id,web_url,url,number,state,cancel_reason,blocked,blocked_state,jobs
0,0192f6ab-038e-4e77-983b-b36f78e9a955,https://buildkite.com/vllm/ci-aws/builds/10683,https://api.buildkite.com/v2/organizations/vll...,10683,running,None,False,,[{'id': '0192f6ab-03cc-424b-800c-0b85592bf40c'...
1,0192f689-d3dd-4ad7-a037-e96c59255e60,https://buildkite.com/vllm/ci-aws/builds/10682,https://api.buildkite.com/v2/organizations/vll...,10682,passed,None,True,,[{'id': '0192f689-d40e-45a0-9325-bd3147b9105f'...
2,0192f662-2569-4023-a559-5446e984a30f,https://buildkite.com/vllm/ci-aws/builds/10681,https://api.buildkite.com/v2/organizations/vll...,10681,failing,None,False,,[{'id': '0192f662-259a-4579-80fc-ca57f3472504'...
3,0192f648-1a14-4a41-b5c5-09c34bef6169,https://buildkite.com/vllm/ci-aws/builds/10680,https://api.buildkite.com/v2/organizations/vll...,10680,passed,None,True,,[{'id': '0192f648-1a48-4b85-b21f-ae91def0f4d0'...
4,0192f638-5712-4b08-801c-ed9e6b89f128,https://buildkite.com/vllm/ci-aws/builds/10679,https://api.buildkite.com/v2/organizations/vll...,10679,passed,None,True,,[{'id': '0192f638-5744-47b7-b500-ac764c8189c9'...


In [46]:
jobs_df = pd.json_normalize(df['jobs'].explode())
jobs_df.head()

,id,graphql_id,type,name,step_key,agent_query_rules,state,build_url,web_url,log_url,raw_log_url,artifacts_url,command,soft_failed,exit_status,artifact_paths,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,retried_in_job_id,retries_count,retry_source,retry_type,parallel_group_index,parallel_group_total,matrix,cluster_id,cluster_url,cluster_queue_id,cluster_queue_url,step.id,step.signature,priority.number,agent.id,agent.url,agent.web_url,agent.name,agent.connection_state,agent.ip_address,agent.hostname,agent.user_agent,agent.version,agent.creator,agent.created_at,agent.job,agent.last_job_finished_at,agent.priority,agent.meta_data,agent.cluster_url,agent.cluster_queue_url,label,unblocked_by,unblocked_at,unblockable,unblock_url,agent,retry_source.job_id,retry_source.retry_type
0,0192f6ab-03cc-424b-800c-0b85592bf40c,Sm9iLS0tMDE5MmY2YWItMDNjYy00MjRiLTgwMGMtMGI4NT...,script,bootstrap,bootstrap,[queue=small_cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,"if [[ -n """" ]]; then VLLM_CI_BRANCH= curl -sSL...",False,0.0,NaN,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.810Z,2024-11-04T10:14:53.296Z,2024-11-04T10:15:03.428Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-03b5-4762-bc27-2cdd132c0b41,NaN,0.0,0192f6ab-93dd-4cc5-9c6b-e51e811da177,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-small-cpu-queue-i-0e3136c019779f5e9-1,disconnected,54.184.82.161,ip-10-0-110-146.us-west-2.compute.internal,buildkite-agent/3.73.1.8506 (linux; amd64),3.73.1,NaN,2024-11-04T10:14:52.637Z,NaN,2024-11-04T10:15:03.439Z,0.0,"[queue=small_cpu_queue, docker=25.0.3, stack=b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0192f6ab-b82d-49bf-88ab-f0e8a4ccc227,Sm9iLS0tMDE5MmY2YWItYjgyZC00OWJmLTg4YWItZjBlOG...,script,:docker: build image,image-build,[queue=cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,aws ecr-public get-login-password --region us-...,False,0.0,NaN,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:15:03.517Z,2024-11-04T10:15:35.936Z,2024-11-04T10:36:36.578Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-b77a-466e-99df-058598a63151,NaN,0.0,0192f6ac-39a3-4201-96a4-438a1af70dd8,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-cpu-queue-i-090da303a61bda67e-1,disconnected,34.222.117.105,ip-10-0-31-18.us-west-2.compute.internal,buildkite-agent/3.73.1.8506 (linux; amd64),3.73.1,NaN,2024-11-04T10:15:35.075Z,NaN,2024-11-04T10:36:36.587Z,0.0,"[queue=cpu_queue, docker=25.0.3, stack=bk-cpu-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0192f6ab-b82f-4ef4-a469-3908ddd58bf6,Sm9iLS0tMDE5MmY2YWItYjgyZi00ZWY0LWE0NjktMzkwOG...,script,Documentation Build,None,[queue=small_cpu_queue],passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,https://api.buildkite.com/v2/organizations/vll...,,False,0.0,NaN,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:12.163Z,2024-11-04T10:40:18.092Z,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0192f6ab-b77b-4a38-a6c2-6968943c26fe,NaN,0.0,0192f6c0-01b1-4681-8ba4-c2285b73e043,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/organizations/vllm/unclu...,bk-small-cpu-queue-i-08338b91fe8e5a7a0-1,disconnected,35.91.42.218,ip-10-0-189-148.us-west-2.compute.internal,buildkite-agent/3.73

In [12]:
jobs_df.build_url.nunique()

22

In [48]:
jobs_useful_columns = ['id', 'name', 'state', 'build_url', 'web_url', 'soft_failed', 'created_at', 'scheduled_at', 'runnable_at', 'started_at',	'finished_at', 'expired_at', 'retried', 'agent.id', 'agent.name', 'agent.web_url', 'agent.connection_state', 'agent.meta_data']

In [49]:
j = jobs_df[jobs_useful_columns]
j.head()

,id,name,state,build_url,web_url,soft_failed,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,agent.id,agent.name,agent.web_url,agent.connection_state,agent.meta_data
0,0192f6ab-03cc-424b-800c-0b85592bf40c,bootstrap,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.737Z,2024-11-04T10:14:15.810Z,2024-11-04T10:14:53.296Z,2024-11-04T10:15:03.428Z,NaN,False,0192f6ab-93dd-4cc5-9c6b-e51e811da177,bk-small-cpu-queue-i-0e3136c019779f5e9-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=small_cpu_queue, docker=25.0.3, stack=b..."
1,0192f6ab-b82d-49bf-88ab-f0e8a4ccc227,:docker: build image,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:15:03.517Z,2024-11-04T10:15:35.936Z,2024-11-04T10:36:36.578Z,NaN,False,0192f6ac-39a3-4201-96a4-438a1af70dd8,bk-cpu-queue-i-090da303a61bda67e-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=cpu_queue, docker=25.0.3, stack=bk-cpu-..."
2,0192f6ab-b82f-4ef4-a469-3908ddd58bf6,Documentation Build,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:12.163Z,2024-11-04T10:40:18.092Z,NaN,False,0192f6c0-01b1-4681-8ba4-c2285b73e043,bk-small-cpu-queue-i-08338b91fe8e5a7a0-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=small_cpu_queue, docker=25.0.3, stack=b..."
3,0192f6ab-b831-4d62-956d-d4e2b173b962,"Async Engine, Inputs, Utils, Worker Test",running,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:27.559Z,None,NaN,False,0192f6c0-3d58-45da-8c51-0d2e3dc2701a,bk-gpu-1-queue-i-03f47f29dd045cbd2-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=gpu_1_queue, docker=25.0.3, stack=bk-gp..."
4,0192f6ab-b832-4c73-8bf2-2ac0e09d5ff3,Basic Correctness Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10683...,False,2024-11-04T10:15:01.866Z,2024-11-04T10:14:15.737Z,2024-11-04T10:36:36.667Z,2024-11-04T10:37:28.713Z,2024-11-04T10:47:07.782Z,NaN,False,0192f6c0-41fb-4481-afd0-b34b2002f02f,bk-gpu-1-queue-i-01d0f3441596e4639-1,https://buildkite.com/organizations/vllm/unclu...,disconnected,"[queue=gpu_1_queue, docker=25.0.3, stack=bk-gp..."


In [50]:
j.web_url.values

array(['https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-03cc-424b-800c-0b85592bf40c',
       'https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-b82d-49bf-88ab-f0e8a4ccc227',
       'https://buildkite.com/vllm/ci-aws/builds/10683#0192f6ab-b82f-4ef4-a469-3908ddd58bf6',
       ...,
       'https://buildkite.com/vllm/ci-aws/builds/10663#0192f48a-e570-4105-8333-cb6aa6e5c8ac',
       'https://buildkite.com/vllm/ci-aws/builds/10663#0192f48a-e571-459f-91ff-602846ce1b9a',
       'https://buildkite.com/vllm/ci-aws/builds/10662#0192f486-f507-4efa-a36d-5acd2cdfadca'],
      dtype=object)

In [51]:
result_df = d.drop(columns=['jobs']).merge(j.reset_index(drop=True), left_on='url', right_on='build_url', suffixes=['_build', '_job'], how='outer')
result_df.shape 

(1900, 26)

In [57]:
result_df[(result_df.url=='https://api.buildkite.com/v2/organizations/vllm/pipelines/ci-aws/builds/10663') & (result_df.name.str.contains('AMD'))]

,id_build,web_url_build,url,number,state_build,cancel_reason,blocked,blocked_state,id_job,name,state_job,build_url,web_url_job,soft_failed,created_at,scheduled_at,runnable_at,started_at,finished_at,expired_at,retried,agent.id,agent.name,agent.web_url,agent.connection_state,agent.meta_data
51,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e557-475d-872a-b6f476ba68c3,AMD: :docker: build image,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,False,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:19:56.693Z,2024-11-04T00:19:58.620Z,2024-11-04T00:39:53.539Z,NaN,False,0190a38f-f36c-4168-8350-f5f09c6ab9fd,rocm-framework-build-04-4,https://buildkite.com/organizations/vllm/unclu...,connected,[queue=amd-cpu]
52,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e559-4922-beda-bc6fa1528fd6,AMD: Core Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,False,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:39:53.674Z,2024-11-04T00:40:01.979Z,2024-11-04T00:54:28.450Z,NaN,False,0192df24-0cae-4861-af82-7b8941d35e62,gpud525.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]"
53,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e55a-4f0e-8212-b0c8832a52f8,AMD: Entrypoints Test,failed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,True,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:39:53.674Z,2024-11-04T00:40:00.420Z,2024-11-04T01:24:01.898Z,NaN,False,0192b565-f0f7-43d1-8ce1-6f5e2cf6546e,gpud573.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]"
54,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e55c-4511-ae31-5284c4b6d0f0,AMD: Regression Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,False,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:39:53.674Z,2024-11-04T00:39:58.422Z,2024-11-04T00:45:56.375Z,NaN,False,0192e85e-63f9-49d3-bb43-52e9f61b361f,gpu10eb.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]"
55,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e55e-4b52-9061-ef45c86b99e0,AMD: Engine Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,False,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:39:53.674Z,2024-11-04T00:39:54.495Z,2024-11-04T00:54:11.331Z,NaN,False,0192df1f-4752-4421-9969-066f3352ed25,gpud518.jax.cs.cpe.ice.amd.com-1,https://buildkite.com/organizations/vllm/unclu...,connected,"[queue=amd, docker=true, mi250=true]"
56,0192f48a-c0b2-4cd5-b574-807f34a7fdb1,https://buildkite.com/vllm/ci-aws/builds/10663,https://api.buildkite.com/v2/organizations/vll...,10663.0,passed,None,True,,0192f48a-e560-484d-9698-43a9f24ea3d9,AMD: LogitsProcessor Test,passed,https://api.buildkite.com/v2/organizations/vll...,https://buildkite.com/vllm/ci-aws/builds/10663...,False,2024-11-04T00:19:56.276Z,2024-11-04T00:19:46.980Z,2024-11-04T00:39:53.674Z,2024-11-04T00:39:55.069Z,2024-11-04T00:43:54.402Z,NaN,False,01922256-33b6-47ce-bf67-35d0e1bbd089,gpu6931.jax.cs.cpe.ice.amd.com-1,https://bui